In [18]:
import yfinance as yf
import pandas as pd
import math
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import numpy as np

In [13]:
data = yf.download("SPY", start="2024-03-07", end="2024-03-08",interval="1m")[386:]
data = data.drop(["Adj Close"], axis=1)

[*********************100%%**********************]  1 of 1 completed


In [16]:
# chart = go.Figure(data=[go.Candlestick(x=data.index, open=data.Open, high=data.High, low=data.Low, close=data.Close)])
# chart.show()
data_dollar_bars = pd.DataFrame(columns=["Date", "Open", "High", "Low", "Close", "Volume", "DollarValue"], index=pd.to_datetime([]))
data["DollarValue"] = 0.0
prev_index = None
dollar_metric = 20000000 #20mio
for index, row in data.iterrows():
    dollar_value = ((row.Open + row.Close) / 2) * row.Volume
    data.loc[index, "DollarValue"] = dollar_value
    
for index, row in data.iterrows():
    no_rows = row.DollarValue/ dollar_metric
    iterations = int(no_rows)
    for i in range(iterations + 1):
        data_dollar_bars.loc[len(data_dollar_bars)] = {"Date": index, "Open": row.Open, "High": row.High, "Low": row.Low, "Close": row.Close, "Volume": row.Volume, "DollarValue": dollar_metric}

    data.loc[index,"DollarValue"] = data.loc[index,"DollarValue"] - (iterations * dollar_metric)
    
    if (prev_index is not None and data.loc[prev_index, "DollarValue"] != 0):
        weighted_prev= data.loc[prev_index, "DollarValue"] / (data.loc[prev_index, "DollarValue"]+ data.loc[index, "DollarValue"])
        weighted_current = 1 - weighted_prev 
        weighted_open = row.Open * weighted_current + data.loc[prev_index, "Open"] * weighted_prev
        weighted_high = row.High * weighted_current + data.loc[prev_index, "High"] * weighted_prev
        weighted_low = row.Low * weighted_current + data.loc[prev_index, "Low"] * weighted_prev
        weighted_close = row.Close * weighted_current + data.loc[prev_index, "Close"] * weighted_prev
        weighted_volume = row.Volume * weighted_current + data.loc[prev_index, "Volume"] * weighted_prev
        weighted_index = index if weighted_current > weighted_prev else prev_index

        data_dollar_bars.loc[len(data_dollar_bars)] = {"Date": weighted_index, "Open": weighted_open, "High": weighted_high, "Low": weighted_low, "Close": weighted_close, "Volume": weighted_volume, "DollarValue": dollar_metric}
        data.loc[index, "DollarValue"] = data.loc[index, "DollarValue"] - data.loc[prev_index, "DollarValue"]
        data.loc[prev_index, "DollarValue"] = 0
    prev_index = index


In [17]:
data

,Open,High,Low,Close,Volume,DollarValue
Datetime,,,,,,
2024-03-07 15:57:00-05:00,515.169983,515.190002,514.840027,514.940002,661562,0.000000e+00
2024-03-07 15:58:00-05:00,514.940002,515.159973,514.849976,515.145020,789062,0.000000e+00
2024-03-07 15:59:00-05:00,515.140015,515.260010,514.849976,514.859985,1962522,5.039167e+06


In [8]:
data_dollar_bars

,Date,Open,High,Low,Close,Volume,DollarValue
0,2024-03-07 09:30:00-05:00,513.140015,513.320007,512.719971,512.830078,1.766714e+06,20000000
1,2024-03-07 09:30:00-05:00,513.140015,513.320007,512.719971,512.830078,1.766714e+06,20000000
2,2024-03-07 09:30:00-05:00,513.140015,513.320007,512.719971,512.830078,1.766714e+06,20000000
3,2024-03-07 09:30:00-05:00,513.140015,513.320007,512.719971,512.830078,1.766714e+06,20000000
4,2024-03-07 09:30:00-05:00,513.140015,513.320007,512.719971,512.830078,1.766714e+06,20000000
...,...,...,...,...,...,...,...
3049,2024-03-07 15:59:00-05:00,515.140015,515.260010,514.849976,514.859985,1.962522e+06,20000000
3050,2024-03-07 15:59:00-05:00,515.140015,515.260010,514.849976,514.859985,1.962522e+06,20000000
3051,2024-03-07 15:59:00-05:00,515.140015,515.260010,514.849976,514.859985,1.962522e+06,20000000
3052,2024-03-07 15:59:00-05:00,515.140015,515.260010,514.849976,514.859985,1.962522e+06,20000000
